In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from scipy.special import expit
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn import tree

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [4]:



API_KEY = "8edf5a3557214b83a8d24412250109"
local = "Curitiba"  # pode ser "Curitiba, Brazil" ou latitude,longitude
url = "http://api.weatherapi.com/v1/forecast.json"
params = {
    "key": API_KEY,
    "q": local,
    "lang": "pt",     # opcional: traduz descrições para português
    "aqi": "no"       # opcional: inclui dados de qualidade do ar
}

try:
    response = requests.get(url, params=params)
    response.raise_for_status()
    dados = response.json()
    
    # Exemplo de como acessar alguns dados:
    # temperatura = dados["current"]["temp_c"]
    # descricao = dados["current"]["condition"]["text"]
    # umidade = dados["current"]["humidity"]
    # vento = dados["current"]["wind_kph"]

    # print(f"Tempo em {dados['location']['name']}: {descricao.capitalize()}")
    # print(f"Temperatura: {temperatura} °C")
    # print(f"Umidade: {umidade}%")
    # print(f"Vento: {vento} km/h")
   # print(dados)

except requests.exceptions.HTTPError as err:
    print(f"Erro HTTP: {err} — código: {response.status_code}")
except ValueError:
    print("Erro ao decodificar JSON.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

df = pd.json_normalize(dados)
df

    


,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,current.last_updated_epoch,current.last_updated,current.temp_c,current.temp_f,current.is_day,current.condition.text,current.condition.icon,current.condition.code,current.wind_mph,current.wind_kph,current.wind_degree,current.wind_dir,current.pressure_mb,current.pressure_in,current.precip_mm,current.precip_in,current.humidity,current.cloud,current.feelslike_c,current.feelslike_f,current.windchill_c,current.windchill_f,current.heatindex_c,current.heatindex_f,current.dewpoint_c,current.dewpoint_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph,current.short_rad,current.diff_rad,current.dni,current.gti,forecast.forecastday
0,Curitiba,Parana,Brazil,-25.4167,-49.25,America/Sao_Paulo,1757731055,2025-09-12 23:37,1757730600,2025-09-12 23:30,10.2,50.4,0,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003,6.3,10.1,85,E,1025.0,30.27,0.0,0.0,87,0,8.9,47.9,8.3,46.9,9.7,49.5,7.1,44.7,10.0,6.0,0.0,8.2,13.2,0,0,0,0,"[{'date': '2025-09-12', 'date_epoch': 17576352..."


In [5]:
forecast_list = dados["forecast"]["forecastday"]
df_forecast = pd.json_normalize(forecast_list)
df_forecast


,date,date_epoch,hour,day.maxtemp_c,day.maxtemp_f,day.mintemp_c,day.mintemp_f,day.avgtemp_c,day.avgtemp_f,day.maxwind_mph,day.maxwind_kph,day.totalprecip_mm,day.totalprecip_in,day.totalsnow_cm,day.avgvis_km,day.avgvis_miles,day.avghumidity,day.daily_will_it_rain,day.daily_chance_of_rain,day.daily_will_it_snow,day.daily_chance_of_snow,day.condition.text,day.condition.icon,day.condition.code,day.uv,astro.sunrise,astro.sunset,astro.moonrise,astro.moonset,astro.moon_phase,astro.moon_illumination,astro.is_moon_up,astro.is_sun_up
0,2025-09-12,1757635200,"[{'time_epoch': 1757646000, 'time': '2025-09-1...",21.6,70.8,9.6,49.4,14.2,57.5,9.2,14.8,0.0,0.0,0.0,10.0,6.0,71,0,0,0,0,Sol,//cdn.weatherapi.com/weather/64x64/day/113.png,1000,1.9,06:17 AM,06:09 PM,11:41 PM,09:25 AM,Waning Gibbous,77,1,0


In [6]:
hour_list = dados["forecast"]["forecastday"][0]["hour"]
df_hour = pd.json_normalize(hour_list)
df_hour


,time_epoch,time,temp_c,temp_f,is_day,wind_mph,wind_kph,wind_degree,wind_dir,pressure_mb,pressure_in,precip_mm,precip_in,snow_cm,humidity,cloud,feelslike_c,feelslike_f,windchill_c,windchill_f,heatindex_c,heatindex_f,dewpoint_c,dewpoint_f,will_it_rain,chance_of_rain,will_it_snow,chance_of_snow,vis_km,vis_miles,gust_mph,gust_kph,uv,short_rad,diff_rad,dni,gti,condition.text,condition.icon,condition.code
0,1757646000,2025-09-12 00:00,12.7,54.9,0,6.3,10.1,97,E,1021.0,30.16,0.0,0.0,0.0,83,30,11.9,53.4,11.9,53.4,12.8,55.0,10.0,49.9,0,0,0,0,10.0,6.0,9.0,14.4,0.0,0.00,0.00,0.00,0.00,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003
1,1757649600,2025-09-12 01:00,12.0,53.6,0,6.5,10.4,98,E,1021.0,30.16,0.0,0.0,0.0,82,19,11.0,51.7,11.0,51.7,12.0,53.6,9.0,48.2,0,0,0,0,10.0,6.0,9.1,14.7,0.0,0.00,0.00,0.00,0.00,Céu limpo,//cdn.weatherapi.com/weather/64x64/night/113.png,1000
2,1757653200,2025-09-12 02:00,11.3,52.4,0,7.4,11.9,100,E,1021.0,30.15,0.0,0.0,0.0,79,9,10.0,49.9,10.0,49.9,11.3,52.4,7.8,46.1,0,0,0,0,10.0,6.0,10.6,17.0,0.0,0.00,0.00,0.00,0.00,Céu limpo,//cdn.weatherapi.com/weather/64x64/night/113.png,1000
3,1757656800,2025-09-12 03:00,10.8,51.4,0,6.5,10.4,99,E,1021.0,30.15,0.0,0.0,0.0,77,11,9.5,49.1,9.5,49.1,10.8,51.4,6.9,44.4,0,0,0,0,10.0,6.0,9.3,15.0,0.0,0.00,0.00,0.00,0.00,Céu limpo,//cdn.weatherapi.com/weather/64x64/night/113.png,1000
4,1757660400,2025-09-12 04:00,10.2,50.4,0,6.5,10.4,98,E,1021.0,30.16,0.0,0.0,0.0,80,23,8.8,47.9,8.8,47.9,10.2,50.4,6.9,44.4,0,0,0,0,10.0,6.0,9.6,15.4,0.0,0.00,0.00,0.00,0.00,Céu limpo,//cdn.weatherapi.com/weather/64x64/night/113.png,1000
5,1757664000,2025-09-12 05:00,9.6,49.4,0,5.6,9.0,88,E,1022.0,30.17,0.0,0.0,0.0,85,25,8.4,47.1,8.4,47.1,9.6,49.4,7.3,45.2,0,0,0,0,10.0,6.0,8.4,13.5,0.0,0.00,0.00,0.00,0.00,Céu limpo,//cdn.weatherapi.com/weather/64x64/night/113.png,1000
6,1757667600,2025-09-12 06:00,9.9,49.8,0,5.4,8.6,82,E,1022.0,30.18,0.0,0.0,0.0,85,37,8.8,47.8,8.8,47.8,9.9,49.8,7.6,45.6,0,0,0,0,10.0,6.0,7.4,11.9,0.0,0.00,0.00,0.00,0.00,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003
7,1757671200,2025-09-12 07:00,12.0,53.7,1,6.0,9.7,83,E,1022.0,30.19,0.0,0.0,0.0,75,7,11.1,52.0,11.1,52.0,12.1,53.7,7.8,46.1,0,0,0,0,10.0,6.0,7.0,11.2,0.1,6.74,2.86,5.37,5.32,Sol,//cdn.weatherapi.com/weather/64x64/day/113.png,1000
8,1757674800,2025-09-12 08:00,14.4,57.9,1,6.3,10.1,91,E,1022.0,30.19,0.0,0.0,0.0,66,5,13.9,56.9,13.9,56.9,14.4,57.9,8.0,46.5,0,0,0,0,10.0,6.0,7.2,11.6,0.5,48.99,13.48,43.01,36.71,Sol,//cdn.weatherapi.com/weather/64x64/day/113.png,1000
9,1757678400,2025-09-12 09:00,16.7,62.0,1,6.3,10.1,95,E,1022.0,30.18,0.0,0.0,0.0,58,4,16.7,62.0,16.7,62.0,16.7,62.0,8.4,47.2,0,0,0,0,10.0,6.0,7.2,11.6,1.0,115.87,24.08,105.53,84.75,Sol,//cdn.weatherapi.com/weather/64x64/day/113.png,1000


In [7]:
# Lista apenas com as colunas importantes
colunas_importantes = [
    "time",
    "feelslike_c",
    "humidity",
    "cloud",
    "uv",
    "wind_kph",
    "will_it_rain",
    "chance_of_rain",
]

df_limpo = df_hour[colunas_importantes]
df_limpo.sample(5)

,time,feelslike_c,humidity,cloud,uv,wind_kph,will_it_rain,chance_of_rain
5,2025-09-12 05:00,8.4,85,25,0.0,9.0,0,0
13,2025-09-12 13:00,21.5,43,19,8.4,10.4,0,0
2,2025-09-12 02:00,10.0,79,9,0.0,11.9,0,0
3,2025-09-12 03:00,9.5,77,11,0.0,10.4,0,0
7,2025-09-12 07:00,11.1,75,7,0.1,9.7,0,0


In [8]:
# Cria uma tabela associando estilos de roupa com dados do clima
df_tabela = pd.DataFrame({
    "estilo de roupa": ["casual", "esportivo", "social", "streetwear"],
    "data e hora": df_limpo["time"].values[:4],
    "temperatura": df_limpo["feelslike_c"].values[:4],
    "chance de chuva": df_limpo["chance_of_rain"].values[:4],
    "umidade": df_limpo["humidity"].values[:4],
    "uv": df_limpo["uv"].values[:4],
})

df_tabela

,estilo de roupa,data e hora,temperatura,chance de chuva,umidade,uv
0,casual,2025-09-12 00:00,11.9,0,83,0.0
1,esportivo,2025-09-12 01:00,11.0,0,82,0.0
2,social,2025-09-12 02:00,10.0,0,79,0.0
3,streetwear,2025-09-12 03:00,9.5,0,77,0.0


In [11]:
#subdados para peças de roupas por estilo especifico

subdados_roupas = {
    "casual": {
        "casual para calor": ["camisetas polo", "camisetas básicas de algodão", "camisas de manga curta", "bermudas de sarja", "bermuda jeans"],
        "casual para frio": ["suéter leve", "jaqueta jeans", "jaqueta de sarja", "calça jeans", "calça chino"],
        "casual meia estação": ["camisa xadrez", "camisa de botões leve", "cardigan fino", "calça slim"],
    },
    "esportivo": {
        "esportivo para calor": ["camisetas dry-fit", "regatas, shorts esportivos", "bermudas de tecido leve"],
        "esportivo para frio": ["moletom com capuz", "calça jogger", "calça de moletom", "jaqueta corta-vento"],
        "esportivo meia estação": ["blusa térmica fina", "calça legging", "jaqueta leve de treino"],
    },
    "social": {
        "social para calor": ["camisas de linho", "camisas de manga curta sociais", "calças de alfaiataria leve", "calça chino social"],
        "social para frio": ["blazer estruturado", "suéter de lã", "colete de alfaiataria", "calça de lã fria", "trench coat"],
        "social meia estação": ["trench coat leve", "camisa social slim fit", "calça social reta"],
    },
    "streetwear": {
        "streetwear para calor": ["camisetas oversized", "camisas estampadas leves", "bermudas cargo", "bermudas de moletom"],
        "streetwear para frio": ["hoodies com estampas", "jaquetas bomber", "jaquetas jeans oversized", "calça cargo", "moletom oversized"],
        "streetwear meia estação": ["jaqueta utilitária", "camisa xadrez oversized", "calça jogger estilizada", "jaqueta college"],
    }
}

C:\Users\diona\AppData\Local\Temp\ipykernel_9752\695683968.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.subdados_roupas = pd.DataFrame(subdados_roupas)
